In [1]:
# import os
# import shutil

# source_folder = r'F:\NE\flood shiliang\vad\3'
# destination_folder = r'F:\NE\flood shiliang\vad\5'

# # 确保目标文件夹存在，如果不存在则创建
# if not os.path.exists(destination_folder):
#     os.makedirs(destination_folder)

# # 遍历源文件夹中的所有文件
# for filename in os.listdir(source_folder):
#     if filename.endswith('.json'):
#         source_path = os.path.join(source_folder, filename)
#         destination_path = os.path.join(destination_folder, filename)
#         shutil.move(source_path, destination_path)
#         print(f"移动文件：{source_path} 到 {destination_path}")

# print("移动完成！")

In [30]:
import os
import json
import numpy as np
import cv2
import shutil

from tqdm import tqdm

In [ ]:
import os
import json
import numpy as np
import cv2
import shutil
from tqdm import tqdm

Dataset_Path = r'F:\NE\flood shiliang\vad\labelme2marsk'

# 0-背景，从 1 开始
class_info = [
    {'label': 'ketin', 'type': 'polygon', 'color': 1}, 
    {'label': 'woshi', 'type': 'polygon', 'color': 2},
    {'label': 'chufang', 'type': 'polygon', 'color': 3},
    {'label': 'weishengjian', 'type': 'polygon', 'color': 4},
    {'label': 'yangtai', 'type': 'polygon', 'color': 5},
    {'label': 'ketin', 'type': 'rectangle', 'color': 1, 'thickness': -1},
    {'label': 'woshi', 'type': 'rectangle', 'color': 2, 'thickness': -1},
    {'label': 'chufang', 'type': 'rectangle', 'color': 3, 'thickness': -1},
    {'label': 'weishengjian', 'type': 'rectangle', 'color': 4, 'thickness': -1},
    {'label': 'yangtai', 'type': 'rectangle', 'color': 5, 'thickness': -1}
]

def labelme2mask_single_img(img_path, labelme_json_path):
    img_bgr = cv2.imread(img_path)
    img_mask = np.zeros(img_bgr.shape[:2])  # 创建空白图像 0-背景
    with open(labelme_json_path, 'r', encoding='utf-8') as f:
        labelme = json.load(f)
    for one_class in class_info:  # 按顺序遍历每一个类别
        for each in labelme['shapes']:  # 遍历所有标注，找到属于当前类别的标注
            if each['label'] == one_class['label']:
                if one_class['type'] == 'polygon':  # polygon 多段线标注
                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]
                    # 在空白图上画 mask（闭合区域）
                    img_mask = cv2.fillPoly(img_mask, points, color=one_class['color'])
                elif one_class['type'] == 'line' or one_class['type'] == 'linestrip':  # line 或者 linestrip 线段标注
                    # 获取点的坐标
                    points = [np.array(each['points'], dtype=np.int32).reshape((-1, 1, 2))]
                    # 在空白图上画 mask（非闭合区域）
                    img_mask = cv2.polylines(img_mask, points, isClosed=False, color=one_class['color'],
                                              thickness=one_class['thickness'])
                elif one_class['type'] == 'circle':  # circle 圆形标注
                    points = np.array(each['points'], dtype=np.int32)
                    center_x, center_y = points[0][0], points[0][1]  # 圆心点坐标
                    edge_x, edge_y = points[1][0], points[1][1]  # 圆周点坐标
                    radius = np.linalg.norm(np.array([center_x, center_y] - np.array([edge_x, edge_y]))).astype(
                        'uint32')  # 半径
                    img_mask = cv2.circle(img_mask, (center_x, center_y), radius, one_class['color'],
                                          one_class['thickness'])
                elif one_class['type'] == 'rectangle':  # circle 矩形标注
                    points = np.array(each['points'], dtype=np.int32)
                    top_left = tuple(points[0])
                    bottom_right = tuple(points[1])
                    img_mask = cv2.rectangle(img_mask, top_left, bottom_right, one_class['color'],
                                            one_class.get('thickness', -1))
                else:
                    print('未知标注类型', one_class['type'])
    return img_mask



In [ ]:
os.chdir(Dataset_Path)
os.mkdir('masks')
os.chdir('images')


In [ ]:
for img_path in tqdm(os.listdir()):
    try:
        labelme_json_path = os.path.join('../', 'labelme_jsons', '.'.join(img_path.split('.')[:-1]) + '.json')
        img_mask = labelme2mask_single_img(img_path, labelme_json_path)
        mask_path = img_path.split('.')[0] + '.png'
        cv2.imwrite(os.path.join('../', 'masks', mask_path), img_mask)
    except Exception as E:
        print(img_path, '转换失败', E)

In [64]:
import seedir as sd
sd.seedir(Dataset_Path, style='emoji', depthlimit=1)

📁 labelme2marsk/
├─📁 ann_dir/
├─📁 img_dir/
├─📁 train/
└─📁 val/


In [65]:
os.chdir(Dataset_Path)
os.mkdir('train')
os.mkdir('val')

FileExistsError: [WinError 183] 当文件已存在时，无法创建该文件。: 'train'

In [66]:
len(os.listdir('img_dir'))



172

In [67]:
len(os.listdir('ann_dir'))

172

In [68]:
import random
test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现
folder = 'img_dir'

In [69]:
img_paths = os.listdir(folder)
random.shuffle(img_paths) # 随机打乱

val_number = int(len(img_paths) * test_frac) # 测试集文件个数
train_files = img_paths[val_number:]         # 训练集文件名列表
val_files = img_paths[:val_number]           # 测试集文件名列表

print('数据集文件总数', len(img_paths))
print('训练集文件个数', len(train_files))
print('测试集文件个数', len(val_files))

数据集文件总数 172
训练集文件个数 138
测试集文件个数 34


In [70]:
for each in tqdm(train_files):
    src_path = os.path.join(folder, each)
    dst_path = os.path.join('train', each)
    shutil.move(src_path, dst_path)

100%|██████████████████████████████████████████████████████████████████████████████| 138/138 [00:00<00:00, 9510.58it/s]


In [71]:
for each in tqdm(val_files):
    src_path = os.path.join(folder, each)
    dst_path = os.path.join('val', each)
    shutil.move(src_path, dst_path)

100%|█████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 974.01it/s]


In [72]:
len(os.listdir('train')) + len(os.listdir('val'))

172

In [73]:
shutil.move('train', 'img_dir/train')
shutil.move('val', 'img_dir/val')

'img_dir/val'

In [74]:
folder = 'ann_dir'

In [75]:
os.mkdir('train')
os.mkdir('val')

In [76]:
for each in tqdm(train_files):
    src_path = os.path.join(folder, each.split('.')[0]+'.png')
    dst_path = os.path.join('train', each.split('.')[0]+'.png')
    shutil.move(src_path, dst_path)

100%|██████████████████████████████████████████████████████████████████████████████| 138/138 [00:00<00:00, 8652.96it/s]


In [77]:
for each in tqdm(val_files):
    src_path = os.path.join(folder, each.split('.')[0]+'.png')
    dst_path = os.path.join('val', each.split('.')[0]+'.png')
    shutil.move(src_path, dst_path)

100%|█████████████████████████████████████████████████████████████████████████████████| 34/34 [00:00<00:00, 792.62it/s]


In [78]:
len(os.listdir('train')) + len(os.listdir('val'))

172

In [79]:
shutil.move('train', 'ann_dir/train')
shutil.move('val', 'ann_dir/val')

'ann_dir/val'

^C


'1.10.1+cu113'